# 1. Вытаскиваем ссылки, которые накачали

In [3]:
import pickle
import collections

In [5]:
pwd

'/Users/fulyankin/Documents/massResearch_houses'

In [6]:
import os

path_to_directory = '/Users/fulyankin/Documents/massResearch_houses/data'
files =  os.listdir(path_to_directory)

files = [item for item in files if '.pickle' in item]
files

['CAO.pickle',
 'data_D.pickle',
 'data_N.pickle',
 'data_zao.pickle',
 'sao_5district.pickle',
 'TAO.pickle',
 'uao_9district.pickle',
 'UZAO_10district.pickle']

In [7]:
hrefs_real = [ ]

for file in files:
    with open('data/' + file, 'rb') as f:
        data_new = pickle.load(f)
    
    if str(type(data_new)) == "<class 'collections.defaultdict'>":
        hrefs = []
        for k,v in data_new.items():
            hrefs.extend(v) 
        hrefs_real += list(set(hrefs))
    else:
        hrefs_real += data_new # Файлы Димы
        
print(len(hrefs_real), len(set(hrefs_real)))

hrefs_real = list(set(hrefs_real))

35097 34907


# 2. Качаем основную информацию по квартирам

In [9]:
import requests
from bs4 import BeautifulSoup
import re

# модуль с мелкими функциями для части информации
import cian_info as cian

In [10]:
url = hrefs_real[0]
url

'https://www.cian.ru/sale/flat/225377435/'

Объединяем всё в одну функцию! 

In [13]:
def infa_download(soup):
    infa_dict = {}
    infa_dict.update(cian.get_general_information(soup))
    infa_dict.update(cian.get_about(soup))
    infa_dict.update(cian.get_price(soup))
    infa_dict.update(cian.get_description(soup))
    infa_dict.update(cian.get_square_info(soup))
    infa_dict.update(cian.get_address(soup))
    infa_dict.update(cian.get_metro(soup))
    infa_dict.update(cian.get_title(soup))
    infa_dict.update(cian.get_coord(soup))
    return infa_dict

In [14]:
import time
from requests.exceptions import ConnectionError

all_infa = [ ]

for url in hrefs_real[:10]:  # качаю для первого десятка ссылок
    try:
        time.sleep(1)
        soup = cian.get_soup(url)
    except ConnectionError:
        time.sleep(10)
        soup = cian.get_soup(url)
    
    if soup.find("div", {"data-name": "OfferUnpublished"}):
        print('объявление снято')
        continue
        
    cur_infa = infa_download(soup)
    all_infa.append(cur_infa)

объявление снято
объявление снято


In [15]:
hrefs_real.index(url)

9

In [16]:
len(all_infa)

8

Сохранялка. Сохраняем в формате `info_2000_5000.pickle`, где цифры означают с какого по какой файл скачали.

In [ ]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 
with open('data/info_0_10.pickle', 'wb') as f:
    pickle.dump(hrefs_dict, f)

In [ ]:
# можно проверить всё ли ок сохранилось
with open('data/info_0_10.pickle', 'rb') as f:
    data_new = pickle.load(f)

Поглазеть на данные: 

In [17]:
import pandas as pd
df = pd.DataFrame(all_infa)
df.head()

,lat,long,Аварийность,Балкон/лоджия,Вид из окон,Высота потолков,Газоснабжение,Год постройки,Жилая,Кухня,...,Срок сдачи,Тип дома,Тип жилья,Тип перекрытий,Этаж,адрес,метро,описание,тайтл,цена
0,55.574803,37.587847,Нет,NaN,Во двор,"2,7 м",NaN,1992,"20,8 м²","8,6 м²",...,NaN,Панельный,Вторичка,Железобетонные,1 из 14,"Москва, ЮЗАО, р-н Северное Бутово, Старобитцев...",[Бульвар Дмитрия Донского ⋅ 10 мин. пешком],"Недалеко от метро,после косметического ремонта...","1-комн. квартира, 39 м²",6290000
1,55.7612101689,37.5557740823,NaN,3 балкона,NaN,NaN,NaN,NaN,"66,9 м²","53,1 м²",...,2 кв. 2022,Монолитный,Новостройка,NaN,16 из 21,"Москва, ЦАО, р-н Пресненский, 2-я Звенигородск...","[Улица 1905 года ⋅ 7 мин. пешком, Краснопресн...","Продается 4-комнатная квартира, в строящемся д...","4-комн. квартира, 189,18 м²",116156520
2,55.871056,37.531379,Нет,1 лоджия,Во двор,3 м,NaN,2006,25 м²,16 м²,...,NaN,Монолитный,Вторичка,Железобетонные,13 из 17,"Москва, САО, р-н Западное Дегунино, Коровинско...",[Селигерская ⋅ 10 мин. пешком],Собственник. ЭКСКЛЮЗИВ. НЕ АППАРТАМЕНТЫ!\nПрод...,"2-комн. квартира, 55 м²",10200000
3,55.748579,37.598555,Нет,NaN,NaN,NaN,NaN,2002,NaN,NaN,...,NaN,Монолитный,Вторичка,Железобетонные,8 из 9,"Москва, ЦАО, р-н Арбат, Филипповский пер., 8С1","[Арбатская, Кропоткинская]","Лот 64900; ЖК ""Филипповский"" - Видовая 5-комна...","5-комн. квартира, 256 м²",299000000
4,55.579811,37.67131,Нет,1 лоджия,На улицу и двор,NaN,Центральное,2013,19 м²,15 м²,...,NaN,Блочный,Вторичка,Железобетонные,6 из 17,"Москва, ЮАО, р-н Бирюлево Восточное, Михневска...",[Царицыно ⋅ 15 мин. на транспорте],"Продается квартира 1829 площадью 55,9 кв.м. в ...","1-комн. квартира, 55,9 м²",7747740


In [18]:
df.shape

(8, 31)

In [19]:
df.isnull().sum()

lat                0
long               0
Аварийность        4
Балкон/лоджия      5
Вид из окон        2
Высота потолков    4
Газоснабжение      7
Год постройки      4
Жилая              3
Кухня              2
Лифты              0
Мусоропровод       6
Общая              0
Отделка            7
Отопление          4
Парковка           2
Площадь комнат     4
Подъезды           4
Построен           3
Ремонт             3
Санузел            1
Срок сдачи         6
Тип дома           3
Тип жилья          0
Тип перекрытий     4
Этаж               0
адрес              0
метро              0
описание           0
тайтл              0
цена               0
dtype: int64